In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os; import pandas as pd; import numpy  as np
from pandas.tseries.offsets import *
import statsmodels.formula.api as smf
from glob import glob
from datetime import datetime
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from IPython.display import IFrame
def swish(x): return (K.sigmoid(x) * x)

import seaborn as sns
sns.set(style='white', font_scale=1.3, rc={
    'lines.linewidth': 4,
    'axes.grid': True, 'grid.linestyle': ':',
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.spines.right': True,
    'axes.spines.top': True,
    'axes.edgecolor': '.5',
})

In [3]:
# load packages
import pandas as pd
import pickle
import numpy as np
import keras
from keras import backend as K
from keras.models import load_model, Model
from keras.layers import Flatten, Dense, Dropout, Activation, Input, LSTM, Reshape, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from tensorflow.keras import Input, layers
from tensorflow.keras.layers import LeakyReLU
#from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils

from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

# set random seeds
np.random.seed(1)
tf.random.set_seed(2)


In [4]:
# Set the Random Seed
import random


seed_value = 2020

os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [5]:
lob_data=pd.read_csv('/Users/macri/Desktop/out_1.csv')
lob_data.reset_index(drop=True,inplace=True)
lob_data.drop(['index'], axis=1,inplace=True)
ld=lob_data.values.tolist()

In [6]:
def OF(ld):
    of=np.zeros((len(ld),6))
    for i in range(1,7,2):
        for j in range(1,len(lob_data)):
            #BID
            if ld[j-1][i]>ld[j][i] :
                of[j][i]=ld[j][i]
            elif ld[j-1][i]<ld[j][i]:
                of[j][i]=-1*ld[j][i]
            else :
                of[j][i]=ld[j][i]-ld[j-1][i]
            #ASK
            if ld[j-1][i-1]<ld[j][i-1] :
                of[j][i-1]=ld[j][i-2]
            elif ld[j-1][i-1]>ld[j][i-2]:
                of[j][i-1]=-1*ld[j][i-2]
            else :
                of[j][i-1]=ld[j][i-2]-ld[j-1][i-2]
    return of

In [7]:
a=OF(ld)
bCols=np.ravel([['aOF_%d'%level, 'bOF_%d'%level] for level in [1,2,3]])
of_data=pd.DataFrame(a, columns=bCols)
of_data

,aOF_1,bOF_1,aOF_2,bOF_2,aOF_3,bOF_3
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,4004.0,-311.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1048570,-1323.0,-1400.0,900.0,300.0,-200.0,500.0
1048571,-100.0,356.0,-1044.0,0.0,0.0,-1510.0
1048572,0.0,300.0,-56.0,237.0,-63.0,1500.0
1048573,732.0,-527.0,227.0,-1523.0,1286.0,0.0


In [8]:
from scipy.stats.mstats import winsorize
dz=of_data.apply(lambda x: (x-x.mean())/x.std())
of1=dz.to_numpy()
#=of_data.to_xarray()
of=winsorize(of1, limits=(0.05, 0.05))
ord_flo=pd.DataFrame(of,columns=['bOF1','aOF1','bOF2','aOF2','bOF3','aOF3'])
ord_flo

,bOF1,aOF1,bOF2,aOF2,bOF3,aOF3
0,-0.139432,-0.020919,-0.088034,-0.005173,-0.132653,0.051854
1,-0.139432,0.723010,-0.390074,-0.005173,-0.132653,0.051854
2,-0.139432,-0.020919,-0.088034,-0.005173,-0.132653,0.051854
3,-0.139432,-0.020919,-0.088034,-0.005173,-0.132653,0.051854
4,-0.139432,-0.020919,-0.088034,-0.005173,-0.132653,0.051854
...,...,...,...,...,...,...
1048570,-0.664797,-0.543032,0.723010,0.212044,-0.395440,0.506511
1048571,-0.247915,0.111847,-0.664797,-0.005173,-0.132653,-0.664797
1048572,-0.139432,0.090963,-0.142421,0.166428,-0.215431,0.723010
1048573,0.654665,-0.217457,0.132426,-0.664797,0.723010,0.051854


In [10]:
def midPrice(lob_data):
    a=lob_data['AP1'].values.tolist()
    b=lob_data['BP1'].values.tolist()
    m=np.zeros(len(a))
    for i in range(len(a)):
        m[i]=(a[i]+b[i])/2
    return m
def upDown(m):
    u_d=np.zeros(len(a))
    for i in range(1,len(a)):
        if m[i-1]>m[i]:#scende il prezzo
            u_d[i-1]=-1
        elif m[i-1]<m[i]:#sale il prezzo
            u_d[i-1]=1
        else:
            u_d[i-1]=0
    return u_d

In [11]:
m= midPrice(lob_data)
ud=upDown(m)
ord_flo['ud']=ud
ord_flo

,bOF1,aOF1,bOF2,aOF2,bOF3,aOF3,ud
0,-0.139432,-0.020919,-0.088034,-0.005173,-0.132653,0.051854,0.0
1,-0.139432,0.723010,-0.390074,-0.005173,-0.132653,0.051854,0.0
2,-0.139432,-0.020919,-0.088034,-0.005173,-0.132653,0.051854,0.0
3,-0.139432,-0.020919,-0.088034,-0.005173,-0.132653,0.051854,0.0
4,-0.139432,-0.020919,-0.088034,-0.005173,-0.132653,0.051854,0.0
...,...,...,...,...,...,...,...
1048570,-0.664797,-0.543032,0.723010,0.212044,-0.395440,0.506511,-1.0
1048571,-0.247915,0.111847,-0.664797,-0.005173,-0.132653,-0.664797,0.0
1048572,-0.139432,0.090963,-0.142421,0.166428,-0.215431,0.723010,-1.0
1048573,0.654665,-0.217457,0.132426,-0.664797,0.723010,0.051854,0.0


In [12]:
V = int(len(ord_flo)*0.6)
T = int(len(ord_flo)*0.8)
ds_trai = ord_flo.iloc[ :V].copy()
ds_vali = ord_flo.iloc[V:T].copy()
ds_test = ord_flo.iloc[T: ].copy()

In [13]:
def prepare_x(data):
    df1 = data.to_numpy()[:,:6] 
    return np.array(df1)

def get_label(data):
    lob = data.to_numpy()[:,-1]
    return lob
def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)
    dY = np.array(Y)
    dataY = dY[T - 1:N]
    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]
    return dataX.reshape(dataX.shape + (1,)), dataY

def prepare_x_y(data, k, T):
    x = prepare_x(data)
    y = get_label(data)
    x, y = data_classification(x, y, T=T)
    #y = y[:] - 1
    y = np_utils.to_categorical(y, 3)
    return x, y

In [14]:
from keras.utils import np_utils
k = 4 # which prediction horizon
T = 100 # the length of a single input
n_hiddens = 16
trainX_CNN, trainY_CNN = prepare_x_y(ds_trai, k, T)
valX_CNN, valY_CNN = prepare_x_y(ds_vali, k, T)
testX_CNN, testY_CNN=prepare_x_y(ds_test,k,T)
print(trainX_CNN.shape, trainY_CNN.shape)
print(valX_CNN.shape, valY_CNN.shape)
print(testX_CNN.shape, testY_CNN.shape)

(629046, 100, 6, 1) (629046, 3)
(209616, 100, 6, 1) (209616, 3)
(209616, 100, 6, 1) (209616, 3)


In [ ]:
#rete neurale

def CNN_LSTM(time_series, lag_param, dropout):
    
    # Convolutions across LOB levels
    if time_series == 'OF':
        input_tensor = Input(shape=(lag_param,20,1))
        # Combine imbalance information across sides for each level of the order book
        layer_x = layers.Conv2D(filters=16, kernel_size=(1,2), strides=(1,2))(input_tensor)
        layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
        # Combine imbalance information across time for each side and level of the order book
        layer_x = layers.Conv2D(filters=16, kernel_size=(4,1), padding='same')(layer_x)
        layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
        layer_x = layers.Conv2D(filters=16, kernel_size=(4,1), padding='same')(layer_x)
        layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    elif time_series == 'OFI':
        input_tensor = Input(shape=(lag_param,10,1))
        # Combine imbalance information across time for each side and level of the order book
        layer_x = layers.Conv2D(filters=16, kernel_size=(4,1), padding='same')(input_tensor)
        layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
        layer_x = layers.Conv2D(filters=16, kernel_size=(4,1), padding='same')(layer_x)
        layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    else:
        raise Exception("'time_series' should be 'OF' or 'OFI'")

    # Combine imbalance information across all levels of the book
    layer_x = layers.Conv2D(filters=16, kernel_size=(1,10))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    # Inception Module
    # Tower 1
    tower_1 = layers.Conv2D(filters=32, kernel_size=(1,1), padding='same')(layer_x)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = layers.Conv2D(filters=32, kernel_size=(3,1), padding='same')(tower_1)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
    # Tower 2
    tower_2 = layers.Conv2D(filters=32, kernel_size=(1,1), padding='same')(layer_x)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = layers.Conv2D(filters=32, kernel_size=(5,1), padding='same')(tower_2)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  
    # Tower 3
    tower_3 = layers.MaxPooling2D(pool_size=(3,1), padding='same', strides=(1,1))(layer_x)
    tower_3 = layers.Conv2D(filters=32, kernel_size=(1,1), padding='same')(tower_3)
    tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)

    # Concatenation and reshaping
    layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)
    layer_x = layers.Reshape(target_shape=(lag_param, 96))(layer_x)
    
    # Insert variational dropout layer
    # By setting training to true, we enable dropout during evaluation passes
    layer_x = layers.Dropout(dropout)(layer_x, training=True)
    
    # LSTM with 64 hidden units
    layer_x = layers.LSTM(units=64)(layer_x)
    
    # Final output layer
    output = layers.Dense(units=3, activation='softmax')(layer_x)
    
    model = Model(input_tensor, output)
    
    opt = Adam(learning_rate=0.01, epsilon=1)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [20]:
def create_nn(T, NF, number_of_lstm):
    input_tensor = Input(shape=(T,NF,1))
    # Combine imbalance information across sides for each level of the order book
    layer_x = layers.Conv2D(filters=16, kernel_size=(1,2), strides=(1,2))(input_tensor)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    # Combine imbalance information across time for each side and level of the order book
    layer_x = layers.Conv2D(filters=16, kernel_size=(4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(filters=16, kernel_size=(4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    # Combine imbalance information across all levels of the book
    #layer_x = layers.Conv2D(filters=16, kernel_size=(1,10))(layer_x)
    #layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    # Inception Module
    # Tower 1
    tower_1 = layers.Conv2D(filters=32, kernel_size=(1,1), padding='same')(layer_x)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = layers.Conv2D(filters=32, kernel_size=(3,1), padding='same')(tower_1)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
    # Tower 2
    tower_2 = layers.Conv2D(filters=32, kernel_size=(1,1), padding='same')(layer_x)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = layers.Conv2D(filters=32, kernel_size=(5,1), padding='same')(tower_2)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  
    # Tower 3
    tower_3 = layers.MaxPooling2D(pool_size=(3,1), padding='same', strides=(1,1))(layer_x)
    tower_3 = layers.Conv2D(filters=32, kernel_size=(1,1), padding='same')(tower_3)
    tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)

    # Concatenation and reshaping
    layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)
    layer_x = layers.Reshape(target_shape=(T, 3, 96))(layer_x)
    
    # Insert variational dropout layer
    # By setting training to true, we enable dropout during evaluation passes
    #layer_x = layers.Dropout(dropout)(layer_x, training=True)
    
    # LSTM with 64 hidden units
    layer_x = layers.LSTM(units=64)(layer_x)
    
    # Final output layer
    output = layers.Dense(units=3, activation='softmax')(layer_x)
    
    model = Model(input_tensor, output)
    
    opt = Adam(learning_rate=0.01, epsilon=1)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model    
deeplob = create_nn(trainX_CNN.shape[1], trainX_CNN.shape[2], n_hiddens)
deeplob.summary()
checkpoint_filepath = './model_tensorflow2/weights'
deeplob.compile(  loss='mse',  optimizer='adam')
deeplob.save(filepath='-CNN-LSTM-SAVED.hdf5',overwrite=True)

ValueError: Input 0 of layer "lstm_2" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 100, 3, 96)